## Random Forest Regression Model for Points, Rebounds, Assists Prediction

The following model pulls from the nba schema specifically the player_stats table to individually predict points, rebounds and assists. Expected Points Added (EPA) feature was engineered to give the model more context for the performance of each player and the overall impact on the team per possesion. Active numeric converts the boolean variable 'Active' to a binary value. String variables are dropped, then the sets for predictions are created.

In [22]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy import Engine

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../')))

from data.core.db import engine_nba

today = f'{str(pd.Timestamp.today().date()).replace("-", "")}'

with engine_nba.connect() as conn:
    preds = pd.read_sql(sql = 'SELECT * FROM nba.player_stats LIMIT 200000', con=conn)
    preds = preds.pivot(index=['game_id', 'team_id', 'player_id', 'player_name'], columns='variable', values='value').reset_index()
    #player_names_df = pd.read_sql(sql="SELECT DISTINCT player_name FROM nba.player_stats", con=conn)

prop_data_df = pd.DataFrame(preds)

#convert target variable to numeric
prop_data_df['statistics_points'] = pd.to_numeric(prop_data_df['statistics_points'], errors='coerce')
prop_data_df['statistics_field_goals_att'] = pd.to_numeric(prop_data_df['statistics_field_goals_att'], errors='coerce')
prop_data_df['statistics_field_goals_pct'] = pd.to_numeric(prop_data_df['statistics_field_goals_pct'], errors='coerce')

# Expected Points Added calculation by player
prop_data_df['EPA'] = (prop_data_df['statistics_points'] - 
                       2 * prop_data_df['statistics_field_goals_att'] * 
                       (prop_data_df['statistics_field_goals_pct'] / 100)
                       )

# drop unnessecary columns
#prop_data_df.drop(columns= 'ejected')
#Handle categorical variables
prop_data_df['active_numeric'] = prop_data_df['active'].apply(lambda x: 1 if str(x).lower() == 'true' else 0)



In [23]:
# subesetting data to form response and predictor variables
reference_columns = ['player_id', 'game_id', 'team_id', 'player_name']
target_columns = ['statistics_points', 'statistics_rebounds', 'statistics_assists']
irrelevant_columns = ['active' , 'on_court' , 'first_name' , 
                      'last_name' , 'name_suffix' , 'fouled_out' , 'jersey_number',
                     'reference' , 'sr_id' , 'starter', 'position', 'primary_position',
                      'played', 'not_playing_description', 'not_playing_reason',
                      'statistics_double_double', 'statistics_triple_double','statistics_minutes']

X = prop_data_df.drop(columns = reference_columns + target_columns + irrelevant_columns)
X = X.fillna(0)
X = X.apply(pd.to_numeric,errors = 'coerce')
Y = prop_data_df[target_columns]
Y = Y.fillna(0)
Y = Y.apply(pd.to_numeric, errors = 'coerce')


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)

predictions_test = rf_model.predict(X_test)
predictions_train = rf_model.predict(X_train)


# Curate dataframe and display results
test_results = pd.DataFrame({
    'player_name': prop_data_df.loc[X_test.index, 'player_name'],
    'player_id': prop_data_df.loc[X_test.index, 'player_id'],
    'game_id': prop_data_df.loc[X_test.index, 'game_id'],
    'team_id': prop_data_df.loc[X_test.index, 'team_id'],
    'predicted_points': predictions_test[:, 0],
    'predicted_rebounds': predictions_test[:, 1],
    'predicted_assists': predictions_test[:, 2]
})


train_results = pd.DataFrame({
    'player_name': prop_data_df.loc[X_train.index, 'player_name'],
    'player_id': prop_data_df.loc[X_train.index, 'player_id'],
    'game_id': prop_data_df.loc[X_train.index, 'game_id'],
    'team_id': prop_data_df.loc[X_train.index, 'team_id'],
    'predicted_points': predictions_train[:, 0],
    'predicted_rebounds': predictions_train[:, 1],
    'predicted_assists': predictions_train[:, 2]
})



full_results = pd.concat([test_results, train_results], ignore_index= True)

full_results.to_csv('app/grant/EPA_model_results.csv', index = False)




In [25]:
from sklearn.metrics import mean_absolute_error, r2_score

# basic model diagnostics
print(f'Mean absolute error for testing set: {mean_absolute_error(Y_test, predictions_test)}')
print(f'Mean absolute error for training set: {mean_absolute_error(Y_train, predictions_train)}')
print(f'R-Squared value for testing set: {r2_score(Y_test, predictions_test)}')
print(f'R-Squared value for training set: {r2_score(Y_train, predictions_train)}')


Mean absolute error for testing set: 0.4598075834748157
Mean absolute error for training set: 0.1744732370433305
R-Squared value for testing set: 0.8243496953274055
R-Squared value for training set: 0.9771846053434703


In [26]:
from sklearn.model_selection import GridSearchCV

# Model fine tuning to attempt to increase performance
parameters = {'n_estimators' : [125, 130, 135],
              'max_depth' : [18, 22, 24], 
              'max_features' : [10, 11, 12],
              'max_leaf_nodes' : [86, 88, 90]
                }

cv_model = RandomForestRegressor()
cv_model = GridSearchCV(cv_model, parameters)

cv_model.fit(X_train, Y_train)

cv_model_prediction_train = cv_model.predict(X_train)
cv_model_prediction_test = cv_model.predict(X_test)


# Display fine-tuned model diagnostics
print(f'Mean absoulte error for tuned training set: {mean_absolute_error(Y_train,cv_model_prediction_train)}')
print(f'Mean absoulte error for tuned testing set: {mean_absolute_error(Y_test,cv_model_prediction_test)}')
print(f'R-Squared value for tuned training set: {r2_score(Y_train, cv_model_prediction_train)}')
print(f'R-Squared value for tuned testing set: {r2_score(Y_test, cv_model_prediction_test)}')
print(f'Best parameters for prediction: {cv_model.best_params_}')



Mean absoulte error for tuned training set: 0.5009035606369645
Mean absoulte error for tuned testing set: 0.591901461640491
R-Squared value for tuned training set: 0.8618244067192314
R-Squared value for tuned testing set: 0.7788995010814249
Best parameters for prediction: {'max_depth': 18, 'max_features': 12, 'max_leaf_nodes': 88, 'n_estimators': 135}


## Conclusion 

The original untuned model outperforms the tuned model with adjusted parameters in terms of both MAE and R^2. Therefore I reccomend basing predictions off the full_results dataframe. Next steps are to creating a baseline model using linear regression to have a way to compare predictions made using the Random Forest Regression model. 

In [28]:

with engine_nba.connect() as conn:
    preds = pd.read_sql(sql = 'SELECT * FROM nba.model_predictions LIMIT 200000', con=conn)
    preds = preds.pivot(index='game_id', columns='variable', values='value').reset_index()

preds.head(10)



ValueError: Index contains duplicate entries, cannot reshape